## Vom DataFrame zum GPT 3.5/4.0 Prompt

In [6]:
from openai import OpenAI
import csv
import pandas as pd
import json
from sklearn.model_selection import train_test_split

In [2]:
df=pd.read_csv("stripped_train.csv")

In [3]:
X_train, X_test, y_train, y_test = train_test_split(df["source"], df["target"], test_size=0.2, random_state=42)

In [4]:
#train_df=pd.read_csv("Data.csv")
train=pd.DataFrame(columns=["source","target"])
train["source"]=X_train.reset_index(drop=True)
train["target"]=y_train.reset_index(drop=True)
test=pd.DataFrame(columns=["source","target"])
test["source"]=X_test.reset_index(drop=True)
test["target"]=y_test.reset_index(drop=True)

In [5]:
source_list=train["source"].to_list()
target_list=train["target"].to_list()
test_source_list=test["source"].to_list()
test_target_list=test["target"].to_list()

In [6]:
def df_to_dict(diction, s_list, t_lists):
    for i in range(len(s_list)):
        diction[i]=[]
        diction[i].append({"messages":[{"role":"system","content":"Hier soll eine EDI Bestellung ins IDOC Format uebersetzt werden"},
                                       {"role":"user","content":s_list[i]},{"role":"assistant","content":t_lists[i]}]})

In [7]:

train_dict = {}
test_dict={}
df_to_dict(train_dict,source_list, target_list)
df_to_dict(test_dict,test_source_list,test_target_list)

In [8]:
values_train = list(train_dict.values())
values_test = list(test_dict.values())

In [9]:
len(values_train)

9007

In [10]:
def turn_into_jsonl_gpt35(string,lists):
    with open(string, 'w') as file:
            for value in range(len(lists)):
            #for value in range(50):
                json_line=json.dumps(lists[value][0])
                file.write(json_line+ '\n')
                    # Die Werte als JSON-String in einer Zeile speichern
                    #json_line = json.dumps(value)
                    #file.write(json_line + '\n')

In [11]:
turn_into_jsonl_gpt35("trainchat_all.jsonl",values_train)
turn_into_jsonl_gpt35("validchat_all.jsonl",values_test)


In [11]:
import tiktoken
import json
from tqdm import tqdm


def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613"):
    """Return the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        print("Warning: model not found. Using cl100k_base encoding.")
        encoding = tiktoken.get_encoding("cl100k_base")
    if model in {
        "gpt-3.5-turbo-0613",
        "gpt-3.5-turbo-16k-0613",
        "gpt-4-0314",
        "gpt-4-32k-0314",
        "gpt-4-0613",
        "gpt-4-32k-0613",
        }:
        tokens_per_message = 3
        tokens_per_name = 1
    elif model == "gpt-3.5-turbo-0301":
        tokens_per_message = 4  # every message follows <|start|>{role/name}\n{content}<|end|>\n
        tokens_per_name = -1  # if there's a name, the role is omitted
    elif "gpt-3.5-turbo" in model:
        print("Warning: gpt-3.5-turbo may update over time. Returning num tokens assuming gpt-3.5-turbo-0613.")
        return num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613")
    elif "gpt-4" in model:
        print("Warning: gpt-4 may update over time. Returning num tokens assuming gpt-4-0613.")
        return num_tokens_from_messages(messages, model="gpt-4-0613")
    else:
        raise NotImplementedError(
            f"""num_tokens_from_messages() is not implemented for model {model}. See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens."""
        )
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>
    return num_tokens

# Number of tokens in the training set
counter = 0

# Load the training set
with open("/Users/huyduc/Documents/GitHub/Hettich/trainchat_all.jsonl", "r") as f:
    data = [json.loads(line) for line in f.readlines()]

# Count the jsons
print(f"{len(data)} jsons")

# Count the tokens
for d in tqdm(data):

    counter = counter + (num_tokens_from_messages(d["messages"]))

# Print the resulting number of tokens
print(f"{counter} tokens")
print(f"{(counter*3*0.008)/1000} USD kostet der ganze Spaß.")



9007 jsons


100%|██████████| 9007/9007 [00:23<00:00, 387.82it/s]

50689285 tokens
1216.54284 USD kostet der ganze Spaß.
